## Diagnostics for sCMOS with RQE correction (2d)

### Configure environment

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot

import numpy
import os

os.chdir("/home/hbabcock/Data/storm_analysis/sa_diagnostics/scmos_rqe")
print(os.getcwd())

numpy.random.seed(1)

In [ ]:
import storm_analysis.diagnostics.sCMOS.settings as settings
import storm_analysis.diagnostics.sCMOS.configure as configure
import storm_analysis.diagnostics.sCMOS.make_data as makeData
import storm_analysis.diagnostics.sCMOS.analyze_data as analyzeData
import storm_analysis.diagnostics.sCMOS.collate as collate


In [ ]:
settings.photons = [[20, 225]]

settings.iterations = 20
settings.model = '2dfixed'
settings.n_frames = 20
settings.peak_locations = None
settings.threshold = 8.0

rqe_correction_test = True

In [ ]:
# sCMOS calibration files.
period = 19
rqe_term = 0.92

offset = numpy.zeros((settings.y_size, settings.x_size)) + settings.camera_offset
variance = numpy.ones((settings.y_size, settings.x_size)) * settings.camera_variance


rqe = numpy.ones((settings.y_size, settings.x_size))
t1 = (1.0 + (1 - rqe_term)*numpy.sin(2.0*numpy.pi*numpy.arange(rqe.shape[1])/period))

for i in range(rqe.shape[0]):
    rqe[i,:] = rqe[i,:] * t1

gain = 2.0*numpy.ones((settings.y_size, settings.x_size))/rqe

cal_file = "calib.npy"
numpy.save(cal_file, [offset, variance, gain, rqe, 2])

pyplot.imshow(rqe)
pyplot.show()

### Configure

In [ ]:
configure.configure(cal_file)

### Make Data

In [ ]:
makeData.makeData(cal_file)

### Analyze data

In [ ]:
# Use constant RQE in calibration file.
if not rqe_correction_test:
    rqe = numpy.ones((settings.y_size, settings.x_size))

    numpy.save(cal_file, [offset, variance, gain, rqe, 2])


In [ ]:
%time analyzeData.analyzeData()

### Collate data

In [ ]:
# If the RQE correction is not working the actual variance will be much larger
# than the expected variance, at least 2x or more. 
collate.collateRQE()

### Reference results